In [2]:
import collections
import random

In [ ]:
PIPELINE_WIDTH = 2
NUM_RESERVATION_STATIONS = 4
NUM_ROB_ENTRIES = 8
NUM_REGISTERS = 32
INSTRUCTION_QUEUE_SIZE = 128
opcodes = {
    0x0 : 'rtype', 0x8: 'addi', 0xd :'ori', 0xe: 'xori', 0xc:'andi', 0xa:'slti',
    0x23: 'lw', 0x2b : 'sw', 0x4:'beq', 0x2:'j', 0x3 : 'jal', 0x5:'bne'
      
}
notWrites = [0x2b, 0x4, 0x2, 0x5]
rtypes = {
    0x20: 'add', 0x22:'sub', 0x24:'and', 0x25: 'or', 0x2a:'slt', 0x26: 'xor', 
    0x27:'nor', 0x0:'sll', 0x2:'srl', 0x8:'jr'
    
}
def signed(data):
        if data & 0x8000:
            return data - (1 << 16)  
        else: return data

In [ ]:
instructions = []
with open("imem.txt", 'r') as inst:
    for line in inst:
        parts = line.split(":")
        if len(parts)>1:
            instructions.append(int(parts[1].strip().rstrip(";"),2))
    

In [33]:
class Instruction:
    def __init__(self, instruction):
        self.instructionVal = bin(instruction)
        self.opcode = instruction >> 26 & 0b111111
        self.rs = instruction >> 21 & 0b11111
        self.rt = instruction >> 16 & 0b11111
        self.rd = instruction >> 11 & 0b11111
        validRt = opcodes[self.opcode] == 'rtype' or opcodes[self.opcode] =='bne' or opcodes[self.opcode] == 'beq'
        self.destreg = inst.rd if validRt else inst.rt
        self.shamt = instruction >> 6 & 0b11111
        self.funct = instruction & 0b111111
        self.imm = instruction & 0xFFFF

In [12]:
class ROBEntry:
    def __init__(self, instruction):
        self.instruction = instruction
        self.value = None
        self.ready = False
        self.destination = instruction.dest

In [17]:
class ReservationStation:
    def __init__(self):
        self.opcode = None
        self.tag_dest = None
        self.tag_src1 = None
        self.tag_src2 = None
        self.value_src1 = None
        self.value_src2 = None
        self.busy = False

In [26]:
class PipelineRegister:
    def __init__(self):
        self.flush = 0
        self.instruction = 0
        self.instructionVal = 0
        self.prediction = 0
        self.stateIndex = 0
        self.last = 0
        self.data = 0 # any data, depending on the stage. signals are interpreted directly w/o a CU so no need for multiple signals 
        self.done = 0
        self.flush = 0
        self.pc = 0

In [11]:
class Register:
    def __init__(self):
        self.value = 0
        self.tag = None  # Tag indicates which ROB entry is producing this value

In [22]:
instruction_memory = []
registers = [Register() for _ in range(NUM_REGISTERS)]
rob = collections.deque(maxlen=NUM_ROB_ENTRIES)
reservation_stations = [ReservationStation() for _ in range(NUM_RESERVATION_STATIONS)]


class ProcessorState:
    def __init__(self):
        self.cycle = 0
        self.ifidFlush = 0
        self.idexFlush = 0
        self.branchflushcount = 0
        self.branchInstruction  = 0
        self.histReg = 0
        self.ifidStall = 0
        self.pc = 0
        self.registers = registers = [Register() for _ in range(NUM_REGISTERS)]
        self.instructionsQueue = []
        self.rob = collections.deque(maxlen = NUM_ROB_ENTRIES)
        self.reservation_stations = [ReservationStation() for _ in range(NUM_RESERVATION_STATIONS)]
        self.memory = [0] * 1024

In [35]:
state = ProcessorState()
IF_ID = PipelineRegister()
ID_DISPATCH = PipelineRegister()
DISPATCH_ISSUE = PipelineRegister()
state.memory[0] = 0x5
state.memory[1] = 0x7
state.memory[2] = 0x2
state.memory[3] = 0xF
state.memory[4] = 0xA
state.memory[5] = 0x10
state.memory[6] = 0x30
state.memory[7] = 0x1
state.memory[8] = 0xFF
state.memory[9] = 0x55

In [30]:
def fetch(state, instructions, IF_ID):
    if IF_ID.done:
        return
    else:
        insts = []
        for i in range(PIPELINE_WIDTH):
            if state.pc <len(instructions):
                insts.append(Instruction(instructions[state.pc]))
            else: 
                IF_ID.last = 1
                inst = Instruction(0)
            IF_ID.thisPC = state.pc
            IF_ID.rs = inst.rs
            IF_ID.rt = inst.rt
            IF_ID.pcp1 = state.pc +1
            if ((opcodes[inst.opcode] == 'beq' or opcodes[inst.opcode] =='bne' )):
                state.pc = state.pc +1+ signed(inst.imm )& 0b1111111111
            elif opcodes[inst.opcode] == 'j' or opcodes[inst.opcode] =='jal':
                state.pc = inst.imm &0b1111111111
            elif opcodes[inst.opcode] == 'rtype' and rtypes[inst.funct] == 'jr':
                state.pc = state.registers[inst.rs] & 0b1111111111
            else: 
                 state.pc += 1
        IF_ID.insts = insts
        IF_ID.nextPC = state.pc
        IF_ID.done = 1
        


In [ ]:
def decode(state, IF_ID, ID_DISPATCH):
    if not IF_ID.done:
        return
    else:
        ID_DISPATCH = IF_ID #Simulate a pipeline stage
        ID_DISPATCH.done = 1





In [40]:
def dispatch(state, ID_DISPATCH, DISPATCH_ISSUE):
    if not ID_DISPATCH.done:
        return
    else:
        insts = ID_DISPATCH.insts
        for i in range(PIPELINE_WIDTH):
            inst = insts.pop(0)
            if len(state.instructionsQueue) < INSTRUCTION_QUEUE_SIZE:
                state.instructionsQueue.append(inst)
        DISPATCH_ISSUE = ID_DISPATCH
        DISPATCH_ISSUE.done = 1
        
            

In [ ]:
def ISSUE(state, DISPATCH_ISSUE):
    if not DISPATCH_ISSUE.done:
        return
